In [41]:
# import models
import mplfinance as mpf
from config import api_key
import requests
import pandas as pd
import time
import datetime
from datetime import time
import matplotlib.pyplot as plt
import cufflinks as cf
import os


In [42]:
#input info used to pass through api 
symbol = input("What ticker do you want to search?")
periodtype = input("Period type (day, month, year)")
frequency = input("Frequency of data")
frequencytype = input("Frequency type (minute, monthly, daily)")
period = input("Period of data")
needExtendedHoursData = input('Extended hours data (True or False)')


What ticker do you want to search?META
Period type (day, month, year)year
Frequency of data1
Frequency type (minute, monthly, daily)monthly
Period of data20
Extended hours data (True or False)False


In [43]:
#url for td
url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)
    
#parameters
key = {'apikey':api_key,
         'periodType':periodtype,
          'frequencyType': frequencytype,
          'frequency': frequency,
          'period': period,
          'needExtendedHoursData': needExtendedHoursData,
         }
#requesting data
content = requests.get(url = url, params=key)

data = content.json()
data

{'candles': [{'open': 42.05,
   'high': 45.0,
   'low': 26.83,
   'close': 29.6,
   'volume': 1259105906,
   'datetime': 1335848400000},
  {'open': 28.892,
   'high': 33.45,
   'low': 25.52,
   'close': 31.095,
   'volume': 667978381,
   'datetime': 1338526800000},
  {'open': 31.25,
   'high': 32.88,
   'low': 21.61,
   'close': 21.71,
   'volume': 520227811,
   'datetime': 1341118800000},
  {'open': 21.5,
   'high': 22.45,
   'low': 18.03,
   'close': 18.058,
   'volume': 1151948402,
   'datetime': 1343797200000},
  {'open': 18.08,
   'high': 23.37,
   'low': 17.55,
   'close': 21.66,
   'volume': 1058639882,
   'datetime': 1346475600000},
  {'open': 22.08,
   'high': 24.25,
   'low': 18.8,
   'close': 21.11,
   'volume': 1101022691,
   'datetime': 1349067600000},
  {'open': 21.08,
   'high': 28.0,
   'low': 18.87,
   'close': 28.0,
   'volume': 1527484362,
   'datetime': 1351746000000},
  {'open': 27.9999,
   'high': 28.88,
   'low': 25.15,
   'close': 26.6197,
   'volume': 119187237

In [44]:
#create arrays for each category
open_tick = []
high_tick = []
low_tick = []
close_tick = []
volume_tick = []
datetime_tick = []
#iterrates through data to append each array with it's value
for i in range(len(data["candles"])):
    open_tick.append(data["candles"][i]["open"])
    high_tick.append(data["candles"][i]["high"])
    low_tick.append(data["candles"][i]["low"])
    close_tick.append(data["candles"][i]["close"])
    volume_tick.append(data["candles"][i]["volume"])
    datetime_tick.append(data["candles"][i]["datetime"])

#converting epoch to datetime       
file = []
for i in range(len(datetime_tick)):
    # divide epoch by 1000 because it is in milliseconds
    test = datetime_tick[i]/1000
    file.append(datetime.datetime.fromtimestamp(test))

#convert to dataframe
theta_df = pd.DataFrame({
            "Open":open_tick,
            "Close":close_tick,
            "High":high_tick,
            "Low":low_tick,
            "Volume":volume_tick,
            "Datetime": file
    })

In [45]:
# set index to Datetime
theta_df = theta_df.set_index("Datetime")

In [46]:
# Calculate % gain for the day 
theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

theta_df

,Open,Close,High,Low,Volume,Gain%
Datetime,,,,,,
2012-04-30 22:00:00,42.050,29.600,45.000,26.83,1259105906,-29.607610
2012-05-31 22:00:00,28.892,31.095,33.450,25.52,667978381,7.624948
2012-06-30 22:00:00,31.250,21.710,32.880,21.61,520227811,-30.528000
2012-07-31 22:00:00,21.500,18.058,22.450,18.03,1151948402,-16.009302
2012-08-31 22:00:00,18.080,21.660,23.370,17.55,1058639882,19.800885
...,...,...,...,...,...,...
2022-01-31 22:00:00,314.555,211.030,328.000,190.22,1128039532,-32.911573
2022-02-28 22:00:00,209.870,222.360,231.151,185.82,733289978,5.951303
2022-03-31 22:00:00,224.550,200.470,236.860,169.00,672300830,-10.723670


In [47]:
# for the stock ticker if a folder does not exist already, one will be created
mypath = f"{symbol}"
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [48]:
# uploads csv to that folder specifying the frequency and type to identify the csv
upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
theta_df.to_csv(upload_file, index = True)